In [1]:
# going to try training the highest performing normalized LSTM model with GCP


# First, import necessary libraries
import tensorflow as tf
import numpy as np
import keras.backend as K

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# setting up the keras stuff
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
from keras.layers import LSTM
# my custom data_utils file
from data_utils_local08 import *

In [ ]:
# interfacing with GCP

# collecting some potentially helpful commands . . . 
# for saving matrices directly to gcp:
#np.save(file_io.FileIO('gs://my-bucket/123', 'w'), np.array([[1,2,3], [4,5,6]]))
    # may also want to try with just plain old np.save command . . . not sure what file_io does
#I also want to learn how to actually train a model using Cloud TPU

In [3]:
# uploading the X values
#X_input = load_Xinput(get_fc6_directory(7)) #VGG16 data

X_input = np.load('../Desktop/InceptionResNetV2_feature_data.npy') #resnet data!
print(X_input.shape)

(7, 212, 248832)


In [4]:
# uploading the Y values
y_data_input = np.load('../Desktop/labels.npy')
print(y_data_input.shape)

(7, 219)


In [5]:
timesteps = X_input.shape[1]
data_dim = X_input.shape[2]

X_train = X_input[:4,:timesteps, :]
Y_train = y_data_input[:4, :timesteps]

X_test = X_input[4:, :timesteps, :]
Y_test = y_data_input[4:, :timesteps]

print(X_test.shape, X_train.shape, Y_train.shape, Y_test.shape)

(3, 212, 248832) (4, 212, 248832) (4, 212) (3, 212)


In [6]:
# THE OLD MODEL IN KERAS

# constructing a many-to-one LSTM model in keras --> inspiration: https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras
# i will start by training a model on only the VGG16 fc6 layer output (that's just one feature)
# should I eventually abstract this LSTM model? Create its own object file?
model = Sequential()

# normalization
model.add(BatchNormalization(input_shape=(timesteps, data_dim)))

model.add(LSTM(212, return_sequences=True))
#input_shape=(timesteps, data_dim)

# going to try adding a flatten layer in here
model.add(Flatten()) # I got this from a github thing, but I still don't completely understand why it works
# add the final dense layer and then softmax
model.add(Dense(212, activation='sigmoid'))
# going to add a softmax activation to this
#model.add(Activation('softmax'))

In [ ]:
# COMPILATION OF THE OLD KERAS MODEL
# compiling LSTM model
# note that Ng used an Adam optimizer and categorical cross-entropy loss
# but this is a binary classification problem so I think the parameters below should suffice
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy', FScore2])

In [7]:
# a new metric of evaluation! the F-score!
def FScore2(y_true, y_pred):
    '''
    The F score, beta=2
    '''
    B2 = K.variable(4)
    OnePlusB2 = K.variable(5)
    pred = K.round(y_pred)
    tp = K.sum(K.cast(K.less(K.abs(pred - K.clip(y_true, .5, 1.)), 0.01), 'float32'), -1)
    fp = K.sum(K.cast(K.greater(pred - y_true, 0.1), 'float32'), -1)
    fn = K.sum(K.cast(K.less(pred - y_true, -0.1), 'float32'), -1)

    f2 = OnePlusB2 * tp / (OnePlusB2 * tp + B2 * fn + fp)

    return K.mean(f2)

In [ ]:
# running the LSTM model
model.fit(X_train, Y_train, epochs = 20, batch_size = 128, validation_data=(X_test, Y_test))
print("finished training!")

Train on 4 samples, validate on 3 samples
Epoch 1/20


In [ ]:
# START OF NEW MODEL THAT IS COMPATIBLE WITH GCP

# setting up Cloud TPU stuff 
# following code from here: https://github.com/tensorflow/tpu/blob/master/models/experimental/cifar_keras/cifar_keras.py

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import flags
import absl.logging as _logging  # pylint: disable=unused-import
import tensorflow as tf

# Cloud TPU Cluster Resolvers
flags.DEFINE_string(
    'tpu', default=None,
    help='The Cloud TPU to use for training. This should be either the name '
    'used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 url.')
flags.DEFINE_string(
    "gcp_project", default=None,
    help="Project name for the Cloud TPU-enabled project. If not specified, we "
    "will attempt to automatically detect the GCE project from metadata.")
flags.DEFINE_string(
    "tpu_zone", default=None,
    help="GCE zone where the Cloud TPU is located in. If not specified, we "
    "will attempt to automatically detect the GCE project from metadata.")

# Model specific paramenters
flags.DEFINE_integer("batch_size", 128,
                     "Mini-batch size for the computation. Note that this "
                     "is the global batch size and not the per-shard batch.")
flags.DEFINE_float("learning_rate", 0.05, "Learning rate.")
flags.DEFINE_string("train_file", "", "Path to cifar10 training data.")
flags.DEFINE_integer("train_steps", 100000,
                     "Total number of steps. Note that the actual number of "
                     "steps is the next multiple of --iterations greater "
                     "than this value.")
flags.DEFINE_bool("use_tpu", True, "Use TPUs rather than plain CPUs")
flags.DEFINE_string("model_dir", None, "Estimator model_dir")
flags.DEFINE_integer("iterations_per_loop", 100,
                     "Number of iterations per TPU training loop.")
flags.DEFINE_integer("num_shards", 8, "Number of shards (TPU chips).")


FLAGS = flags.FLAGS


In [21]:
# trying to view the model output
out = model.predict(X_train_data)
print("model prediction:")
print(out[0])
print("target:")
print(Y_train_data[0])

print("before 64:")
print(out[0][63])
print("64:")
print(out[0][65])

print("rounded")
print(np.round(out)[0])

NameError: name 'X_train_data' is not defined

In [ ]:
#try visualizing this model at some point?